In [ ]:
"""
Comprehensive Python script for:
1) Displaying an image
2) 2D DCT and inverse DCT (visualize coefficients and reconstruction)
3) 2D FFT and inverse FFT (visualize spectrum and reconstruction)
4) Pearson correlation between original and reconstructed images
5) Decide which reconstruction is closer to original
6) Radial brightness spectrum vs frequency
7) Add different kinds of noise (Gaussian, Salt&Pepper, Speckle)
8) Filter the noisy images with several filter types
9) Filter with multiple mask sizes for three filter kinds
10) Evaluate filters by Pearson correlation vs mask size and plot results

Default image: skimage.data.camera(). To use your own image, set IMAGE_PATH.
Outputs: inline plots; optional saving to disk if SAVE_RESULTS=True.

Dependencies: numpy, matplotlib, scipy, scikit-image
Install example: pip install numpy matplotlib scipy scikit-image

Author: Generated by ChatGPT (GPT-5 Thinking mini)
"""

from __future__ import annotations
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy.fft import fft2, ifft2, fftshift
from scipy.signal import wiener
from scipy.ndimage import uniform_filter, median_filter, gaussian_filter
from skimage import data, img_as_float, util
from skimage.color import rgb2gray
from skimage.util import random_noise
import imageio
import warnings

warnings.filterwarnings('ignore')

# --------------------------- CONFIG ---------------------------
IMAGE_PATH = None  # e.g. 'my_photo.jpg' ; set to None to use skimage.data.camera()
SAVE_RESULTS = False  # set True to save plots and filtered images to OUTPUT_DIR
OUTPUT_DIR = 'results'

# Noise parameters (tweak as needed)
GAUSS_VAR = 0.01    # variance for Gaussian additive noise
SP_AMOUNT = 0.05    # proportion for salt & pepper
SPECKLE_VAR = 0.2   # variance for multiplicative (speckle) noise

# Mask sizes to evaluate (odd integers)
MASK_SIZES = [3, 5, 7, 9, 11]

# -------------------------- HELPERS --------------------------

def ensure_outdir(path: str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def load_image(path: str | None):
    if path:
        im = imageio.v2.imread(path)
        if im.ndim == 3:
            im = rgb2gray(im)
        return img_as_float(im)
    return img_as_float(data.camera())


# 2D DCT / IDCT
def dct2(a: np.ndarray) -> np.ndarray:
    return fftpack.dct(fftpack.dct(a.T, norm='ortho').T, norm='ortho')


def idct2(a: np.ndarray) -> np.ndarray:
    return fftpack.idct(fftpack.idct(a.T, norm='ortho').T, norm='ortho')


# Pearson correlation (flattened)
def pearson_corr(a: np.ndarray, b: np.ndarray) -> float:
    a_f = a.ravel()
    b_f = b.ravel()
    a_m = a_f.mean()
    b_m = b_f.mean()
    num = np.sum((a_f - a_m) * (b_f - b_m))
    den = np.sqrt(np.sum((a_f - a_m) ** 2) * np.sum((b_f - b_m) ** 2))
    return float(num / den) if den != 0 else 0.0


# Radial profile (for spectrum vs frequency)
def radial_profile(data: np.ndarray) -> np.ndarray:
    y, x = np.indices(data.shape)
    center = np.array(data.shape) // 2
    r = np.hypot(x - center[1], y - center[0]).astype(np.int32)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / (nr + 1e-12)
    return radialprofile


# Apply three filter types for multiple sizes
def apply_filters_for_sizes(noisy_img: np.ndarray, sizes: list[int]) -> dict:
    results = {'mean': {}, 'gaussian': {}, 'median': {}}
    for s in sizes:
        # mean / box filter
        results['mean'][s] = uniform_filter(noisy_img, size=s)
        # gaussian: sigma scaled from size
        sigma = max(0.3, s / 6.0)
        results['gaussian'][s] = gaussian_filter(noisy_img, sigma=sigma)
        # median: operate on uint8 for stability then scale back
        median_uint8 = median_filter((noisy_img * 255).astype(np.uint8), size=s)
        results['median'][s] = median_uint8.astype(np.float32) / 255.0
    return results


# Evaluate filters: return dict of lists of correlations
def evaluate_filters(original: np.ndarray, filters_dict: dict, sizes: list[int]) -> dict:
    corr_vs_size = {name: [] for name in filters_dict.keys()}
    for name, d in filters_dict.items():
        for s in sizes:
            filtered = d[s]
            corr = pearson_corr(original, np.clip(filtered, 0, 1))
            corr_vs_size[name].append(corr)
    return corr_vs_size


# --------------------------- MAIN ---------------------------

def main():
    img = load_image(IMAGE_PATH)
    h, w = img.shape
    print(f'Image loaded: shape={img.shape}')

    if SAVE_RESULTS:
        ensure_outdir(OUTPUT_DIR)

    # show original
    plt.figure(figsize=(5, 5))
    plt.title('Original image')
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'original.png'), bbox_inches='tight')
    plt.show()

    # ---------------- DCT ----------------
    dct_coeffs = dct2(img)
    recon_dct = idct2(dct_coeffs)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title('DCT coefficients (log magnitude)')
    plt.axis('off')
    plt.imshow(np.log1p(np.abs(dct_coeffs)), cmap='gray')

    plt.subplot(1, 3, 2)
    plt.title('Reconstructed from IDCT')
    plt.axis('off')
    plt.imshow(np.clip(recon_dct, 0, 1), cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Difference (orig - idct)')
    plt.axis('off')
    plt.imshow(img - np.clip(recon_dct, 0, 1), cmap='bwr')
    plt.colorbar(fraction=0.046, pad=0.01)

    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'dct_idct.png'), bbox_inches='tight')
    plt.show()

    # ---------------- FFT ----------------
    fft_coeffs = fft2(img)
    fft_shift = fftshift(fft_coeffs)
    magnitude_spectrum = np.log1p(np.abs(fft_shift))
    recon_fft = np.real(ifft2(fft_coeffs))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title('FFT magnitude spectrum (log)')
    plt.axis('off')
    plt.imshow(magnitude_spectrum, cmap='gray')

    plt.subplot(1, 3, 2)
    plt.title('Reconstructed from iFFT')
    plt.axis('off')
    plt.imshow(np.clip(recon_fft, 0, 1), cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Difference (orig - ifft)')
    plt.axis('off')
    plt.imshow(img - np.clip(recon_fft, 0, 1), cmap='bwr')
    plt.colorbar(fraction=0.046, pad=0.01)

    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'fft_ifft.png'), bbox_inches='tight')
    plt.show()

    # ---------------- Correlations ----------------
    corr_idct = pearson_corr(img, np.clip(recon_dct, 0, 1))
    corr_ifft = pearson_corr(img, np.clip(recon_fft, 0, 1))
    print(f'Pearson correlation original vs IDCT reconstruction: {corr_idct:.6f}')
    print(f'Pearson correlation original vs IFFT reconstruction: {corr_ifft:.6f}')
    closer = 'IDCT' if corr_idct > corr_ifft else 'IFFT'
    print(f'Closer to original by Pearson correlation: {closer}')

    # ---------------- Radial spectrum ----------------
    radial = radial_profile(np.abs(fft_shift))
    freqs = np.arange(len(radial))
    plt.figure(figsize=(6, 4))
    plt.plot(freqs, radial)
    plt.title('Radial brightness spectrum vs frequency')
    plt.xlabel('Radial frequency (pixels)')
    plt.ylabel('Average magnitude')
    plt.grid(True)
    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'radial_spectrum.png'), bbox_inches='tight')
    plt.show()

    # ---------------- Add noises ----------------
    noisy_gauss = random_noise(img, mode='gaussian', var=GAUSS_VAR)
    noisy_sp = random_noise(img, mode='s&p', amount=SP_AMOUNT)
    noisy_speckle = random_noise(img, mode='speckle', var=SPECKLE_VAR)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.title('Gaussian noise'); plt.axis('off'); plt.imshow(noisy_gauss, cmap='gray')
    plt.subplot(1, 3, 2); plt.title('Salt & Pepper'); plt.axis('off'); plt.imshow(noisy_sp, cmap='gray')
    plt.subplot(1, 3, 3); plt.title('Speckle (multiplicative)'); plt.axis('off'); plt.imshow(noisy_speckle, cmap='gray')
    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'noises.png'), bbox_inches='tight')
    plt.show()

    # ---------------- Filtering for mask sizes ----------------
    filters_gauss = apply_filters_for_sizes(noisy_gauss, MASK_SIZES)
    filters_sp = apply_filters_for_sizes(noisy_sp, MASK_SIZES)
    filters_speckle = apply_filters_for_sizes(noisy_speckle, MASK_SIZES)

    # Wiener filter for gaussian noisy image (scipy.signal.wiener)
    wiener_gauss = wiener((noisy_gauss * 255).astype(np.uint8), (5, 5)).astype(np.float32) / 255.0
    wiener_corr = pearson_corr(img, np.clip(wiener_gauss, 0, 1))
    print(f'Wiener filter correlation with original (Gaussian noise): {wiener_corr:.6f}')

    # Visual example of filtering (for one mask size)
    s_example = MASK_SIZES[len(MASK_SIZES) // 2]
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 3, 1); plt.title('Noisy (Gaussian)'); plt.axis('off'); plt.imshow(noisy_gauss, cmap='gray')
    plt.subplot(2, 3, 2); plt.title('Wiener (scipy.signal.wiener)'); plt.axis('off'); plt.imshow(wiener_gauss, cmap='gray')
    plt.subplot(2, 3, 3); plt.title('Mean / Gauss / Median (size=%d)' % s_example); plt.axis('off')
    comb = np.hstack([filters_gauss['mean'][s_example], filters_gauss['gaussian'][s_example], filters_gauss['median'][s_example]])
    plt.imshow(comb, cmap='gray')
    if SAVE_RESULTS:
        plt.savefig(os.path.join(OUTPUT_DIR, 'filters_examples.png'), bbox_inches='tight')
    plt.show()

    # ---------------- Evaluate correlations vs mask size ----------------
    corr_gauss = evaluate_filters(img, filters_gauss, MASK_SIZES)
    corr_sp = evaluate_filters(img, filters_sp, MASK_SIZES)
    corr_speckle = evaluate_filters(img, filters_speckle, MASK_SIZES)

    # Plot helper
    def plot_corr_vs_size(corr_dict: dict, title: str):
        plt.figure(figsize=(6, 4))
        for name, values in corr_dict.items():
            plt.plot(MASK_SIZES, values, marker='o', label=name)
        plt.title(title)
        plt.xlabel('Mask size')
        plt.ylabel('Pearson correlation with original')
        plt.xticks(MASK_SIZES)
        plt.grid(True)
        plt.legend()
        plt.show()

    plot_corr_vs_size(corr_gauss, 'Correlation vs mask size (Gaussian noise)')

    # show wiener correlation as horizontal line
    plt.figure(figsize=(6, 3))
    plt.hlines(wiener_corr, xmin=MASK_SIZES[0], xmax=MASK_SIZES[-1])
    plt.title('Wiener (scipy) correlation (Gaussian noise)')
    plt.ylabel('Pearson correlation')
    plt.show()

    plot_corr_vs_size(corr_sp, 'Correlation vs mask size (Salt & Pepper noise)')
    plot_corr_vs_size(corr_speckle, 'Correlation vs mask size (Speckle / multiplicative noise)')

    # ---------------- Best filter per noise type ----------------
    def best_filter(corr_dict: dict):
        best = {}
        for name, vals in corr_dict.items():
            idx = int(np.argmax(vals))
            best[name] = (MASK_SIZES[idx], vals[idx])
        return best

    best_gauss = best_filter(corr_gauss)
    best_sp = best_filter(corr_sp)
    best_speckle = best_filter(corr_speckle)

    print('Best filters (Gaussian noise) per type -> (mask_size, corr):', best_gauss)
    print('Best filters (Salt&Pepper) per type -> (mask_size, corr):', best_sp)
    print('Best filters (Speckle) per type -> (mask_size, corr):', best_speckle)
    print('Wiener (scipy) correlation for Gaussian noise:', wiener_corr)

    # Visualize best results
    def show_best_results(filters_dict: dict, best_info: dict, title_prefix: str):
        plt.figure(figsize=(12, 4))
        i = 1
        for name, (s, c) in best_info.items():
            plt.subplot(1, 3, i)
            plt.title(f'{title_prefix}\n{name} best size {s}, corr={c:.4f}')
            plt.axis('off')
            img_f = filters_dict[name][s]
            plt.imshow(np.clip(img_f, 0, 1), cmap='gray')
            i += 1
        plt.show()

    show_best_results(filters_gauss, best_gauss, 'Gaussian noise - best')
    show_best_results(filters_sp, best_sp, 'Salt&Pepper - best')
    show_best_results(filters_speckle, best_speckle, 'Speckle - best')

    # --------------- Summary ---------------
    print('\nSummary:')
    print(' - DCT and FFT transforms computed and visualized.')
    print(' - Correlations between original and reconstructions printed.')
    print(' - Noises applied: Gaussian, Salt&Pepper, Speckle.')
    print(' - Filters applied: mean (box), gaussian, median for sizes:', MASK_SIZES)
    print(' - Correlation vs mask size plotted and best masks printed.')

    if SAVE_RESULTS:
        print(f'All plots and selected images saved in folder: {OUTPUT_DIR}')


if __name__ == '__main__':
    main()
